In [1]:
import os
#import pandas as pd 
import modin.pandas as pd # emails went to 236
from tqdm import tqdm
import datetime
from p_tqdm import p_map
from functools import partial
import gc

In [2]:
time_resolution = 21
time_window = 3
lk_path = '/mnt/data0/lkyin/'
c_path = './data_smoothed'+str(time_resolution+time_window)+'/commits/'
e_path = './data_smoothed'+str(time_resolution+time_window)+'/emails/'
mix_path = './data_smoothed'+str(time_resolution+time_window)+'/mix/'

In [3]:
print('reading commits...')
df_data = pd.read_csv(lk_path+'commits_final.csv')
df_data = df_data.loc[df_data.date.str.len()==19]
df_data['day'] = df_data['date'].apply(lambda x: pd.Period(x, freq='D'))
df_data.to_csv("./temp_data.csv", index=False)
print(df_data.dtypes)

reading commits...


To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()



project_name                    object
list_name                       object
date                            object
month                            int64
message_id                      object
sender_name                     object
sender_email                    object
author_name                     object
author_email                    object
file_name                       object
loc                            float64
ref_or_sha                      object
subject                         object
commit_type                     object
author_full_name                object
is_bot                            bool
is_coding                         bool
dealised_author_full_name       object
day                          period[D]
dtype: object


In [4]:
# Use normal pandas here instead
import pandas as pd
df_data = pd.read_csv("./temp_data.csv")
#df_data['day'] = df_data['day'].apply(lambda x: pd.Period(x))
#df_data['day'] = df_data['day'].astype(str)
print('grouping by project...')
df = dict(tuple(df_data.groupby(df_data['project_name'])))
to_path = c_path
if not os.path.exists(to_path):
	os.makedirs(to_path)

grouping by project...


In [5]:
def period_save(df, project):
	daily_df_dict = dict(tuple(df[project].groupby(df[project]['day'])))
	dayint = 0
	for day0, value in daily_df_dict.items():
		day0 = pd.Period(day0)
		#print(day0)
		break
	periodly_df = pd.DataFrame()
	periodnum = 0

	for today in daily_df_dict:		
		daily_df = daily_df_dict[today]
		daily_df = daily_df[daily_df['dealised_author_full_name'].notna()]
		if daily_df.empty: continue
		daily_df['day'] = daily_df['day'].apply(lambda x: pd.Period(x))
		while pd.Period(today) > (day0 + (dayint*pd.Timedelta("1 day"))):
			dayint+=1
		
		if dayint > time_resolution+time_window:
			periodnum+=1
			periodstr = str(periodnum).zfill(3)
			file_path = to_path + '{}__{}.csv'.format(project, periodstr)
			#print("Saving: "+ file_path)
			periodly_df.to_csv(file_path, index=False)
			periodly_df = pd.DataFrame()
			day0+= (time_resolution+time_window)*pd.Timedelta("1 day")
			dayint-= (time_resolution+time_window)

		periodly_df = pd.concat([periodly_df, daily_df], ignore_index=True)
	
	gc.collect()



In [6]:
print('grouping by period...')

for project in tqdm(df):
	period_save(df, project)
print('Commits Done.')

grouping by period...


100%|██████████| 263/263 [24:51<00:00,  5.67s/it]   

Commits Done.


In [7]:
import modin.pandas as pd
print('reading emails...')
df_data = pd.read_csv(lk_path+'emails_final.csv')
df_data = df_data.loc[df_data.date.str.len()==19]
df_data['day'] = df_data['date'].apply(lambda x: pd.Period(x, freq='D'))
df_data.to_csv("./temp_data.csv", index=False)
print(df_data.dtypes)


reading emails...


Data types of partitions are different! Please refer to the troubleshooting section of the Modin documentation to fix this issue.


project_name                    object
list_name                       object
date                            object
month                          float64
message_id                      object
sender_name                     object
references                      object
sender_email                    object
in_reply_to                     object
cc_list                         object
receiver_email                  object
subject                         object
body                            object
from_commit                     object
author_full_name                object
is_bot                            bool
dealised_author_full_name       object
day                          period[D]
dtype: object


In [3]:
import pandas as pd
df_data = pd.read_csv("./temp_data.csv")
print('grouping by project...')
df = dict(tuple(df_data.groupby(df_data['project_name'])))
to_path = e_path
if not os.path.exists(to_path):
	os.makedirs(to_path)
gc.collect()

grouping by project...


0

In [6]:
print('grouping by period...')

for project in tqdm(df):
	period_save(df, project)
print('Emails Done.')

grouping by period...


100%|██████████| 274/274 [09:14<00:00,  2.02s/it]

Emails Done.


In [ ]:
gc.collect()

0